In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool, Tool

llm = ChatOpenAI(
    temperature=0.1,
)

def plus(inputs):
    a, b = inputs.split(",")
    return float(a)+float(b)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        Tool.from_function(
            func=plus, 
            name="Sum calculator", 
            description = "Use this to perform sums of two numbers This tool take two arguments, both should be numbers",
        ),
    ]
)

prompt = "sum of, $355.39, $924.87, $721.2, $1940.29, $573.63, $65.72, $35.00, $552.00, $76.16, $29.12"
# $5,273.38
agent.invoke(prompt)



> Entering new AgentExecutor chain...
I need to find the sum of all these numbers
Action: Sum calculator
Action Input: 355.39, 924.87
Observation: 1280.26
Thought:I need to continue summing the rest of the numbers
Action: Sum calculator
Action Input: 1280.26, 721.2
Observation: 2001.46
Thought:I need to continue summing the rest of the numbers
Action: Sum calculator
Action Input: 2001.46, 1940.29
Observation: 3941.75
Thought:I need to continue summing the rest of the numbers
Action: Sum calculator
Action Input: 3941.75, 573.63
Observation: 4515.38
Thought:I need to continue summing the rest of the numbers
Action: Sum calculator
Action Input: 4515.38, 65.72
Observation: 4581.1
Thought:I need to continue summing the rest of the numbers
Action: Sum calculator
Action Input: 4581.1, 35.00
Observation: 4616.1
Thought:I need to continue summing the rest of the numbers
Action: Sum calculator
Action Input: 4616.1, 552.00
Observation: 5168.1
Thought:I need to continue summing the rest of the n

{'input': 'sum of, $355.39, $924.87, $721.2, $1940.29, $573.63, $65.72, $35.00, $552.00, $76.16, $29.12',
 'output': '$5273.38'}